In [1]:
!pip install -r requirements.txt

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import tool
from langchain_community.utilities import GoogleSerperAPIWrapper

In [3]:
import os

from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

In [4]:
load_dotenv()

azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_deployment = os.getenv("AZURE_DEPLOYMENT_NAME")
azure_api_version = os.getenv("AZURE_API_VERSION")

llm = AzureChatOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
    azure_deployment=azure_deployment,
    api_version=azure_api_version,
    max_tokens=2000,
    temperature=0.1
)

In [5]:
@tool("GoogleSearch")
def search(query_string: str):
    """
    Useful to search for any kinds of information and
    when you need to search the internet for any kinds of information, use this tool.
    Prefer this tool when you search for long queries.
    Should not be used for Article search or Topic Search.
    """

    search = GoogleSerperAPIWrapper()

    return search.run(query_string)

In [6]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
wiki = WikipediaQueryRun(
    name="WikiepdiaSearch",
    description="Use this tool when you want to analyze for information on Wikipedia by Terms, Keywords or any Topics.",
    api_wrapper=api_wrapper)

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [8]:
azure_embedding_deployment = os.getenv(
    "AZURE_EMBEDDING_DEPLOYMENT_NAME", "text-embedding-3-large")

loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
).split_documents(docs)

embeddings = AzureOpenAIEmbeddings(azure_endpoint=azure_endpoint,
                                    model="text-embedding-3-large",
                                    api_key=azure_api_key,
                                    deployment=azure_embedding_deployment,
                                    dimensions=3072,
                                    )
vectordatabase = FAISS.from_documents(documents, embeddings)
retriever = vectordatabase.as_retriever()

In [ ]:
!pip uninstall -y langchain langchain-core langchain-community
!pip install -U langchain langchain-core langchain-community


In [9]:
from langchain_core.tools import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "search for information about langsmith. for any questions related to langsmith, you must use this tool"
)

retriever_tool.name

'langsmith_search'

In [10]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [11]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [12]:
tools = [arxiv, search, wiki, retriever_tool]
tools

[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=1000)),
 StructuredTool(name='GoogleSearch', description='Useful to search for any kinds of information and\nwhen you need to search the internet for any kinds of information, use this tool.\nPrefer this tool when you search for long queries.\nShould not be used for Article search or Topic Search.', args_schema=<class 'langchain_core.utils.pydantic.GoogleSearch'>, func=<function search at 0x7a3146e7fe20>),
 WikipediaQueryRun(name='WikiepdiaSearch', description='Use this tool when you want to analyze for information on Wikipedia by Terms, Keywords or any Topics.', api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/pytho

In [13]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    ("human", "{input}")
])

prompt.messages



[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful AI assistant.'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]

In [15]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    ("human", "{input}")
])

prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful AI assistant.'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]

In [16]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, SystemMessage

agent_executor = create_react_agent(
    llm,
    tools = tools
)

/tmp/ipython-input-3333576309.py:4: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(


In [17]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="tell me about langsmith")
    ]
}):
    print(stream)
    print("***********")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 255, 'total_tokens': 272, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4a331a0222', 'id': 'chatcmpl-Cnoc1NsxLPc6AXCPrrDBp9avmhhDp', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='lc_run--019b2d20-79a2-7ef0-9515-36c8d8

In [18]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="whats the paper 2412.16446 talk about it?")
    ]
}):
    print(stream)
    print("***********")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 264, 'total_tokens': 283, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4a331a0222', 'id': 'chatcmpl-CnocTpKAIuToEEk2sPOqJE6V7Gji2', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='lc_run--019b2d20-e53a-72b1-8e56-992be5

In [19]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="Indian Constitution")
    ]
}):
    print(stream)
    print("***********")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 252, 'total_tokens': 272, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4a331a0222', 'id': 'chatcmpl-Cnocs0JVE1ArpT3DBkMVCQ61Ihywi', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='lc_run--019b2d21-4656-75f1-abe8-1cf111